In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['agg.path.chunksize'] = 10000
sns.set(style="white")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
class PubG:
    def __init__(self):
        files = os.listdir("../input/")
        def transform(i):
            dir_str = "../input/"
            _filepath = dir_str + i
            setattr(self, i.replace('.csv',''), pd.read_csv(_filepath))
        [transform(i) for i in files]
pg = PubG()

In [ ]:
pg.train.head()

# Need to make features related to teams
## I.e. how many team mates does one have?
### How does teamwork relate to winning, are people better as loan wolf?
# Kills
# Distance
# Teamwork

In [ ]:

corr = pg.train[[i for i in pg.train.columns.tolist() if 'Id' not in i]].corr()
mask = np.zeros_like(corr, dtype=np.bool)

f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
pg.train.drop(columns=['roadKills']).plot(kind='density', subplots=True, layout=(5,5), sharex=False)
# plt.show()

In [ ]:
from sklearn import neighbors, datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_squared_error, r2_score

X = pg.train[[i for i in pg.train.columns.tolist() if 'winPlacePerc' not in i and 'Id' not in i]]
y = pg.train[[i for i in pg.train.columns.tolist() if 'winPlacePerc' in i and 'Id' not in i]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

In [ ]:
scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=True)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_predict = lr.predict(X_test)
y_predict[y_predict > 1] = 1
y_predict

In [ ]:
mean_squared_error(y_test, y_predict)

In [ ]:
r2_score(y_test, y_predict)

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


train_sizes, train_scores, valid_scores = learning_curve(LinearRegression(normalize=True), X, y, train_sizes=[50, 80, 110], cv=5)

In [ ]:
train_scores

In [ ]:
valid_scores

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
# cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

In [ ]:
# plot_learning_curve(LinearRegression(normalize=True),title='Learning Curves Linear Regression', X=X, y=y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)